In [ ]:
# Automatic Dataset Download for Heatmap and Correlation Analysis
import os
import requests
import pandas as pd
import numpy as np
from pathlib import Path
import json
from sklearn.datasets import make_classification

def create_heatmap_datasets():
    """Create comprehensive datasets for heatmap and correlation visualization"""
    np.random.seed(456)  # For reproducible heatmaps
    
    data_dir = Path("../data")
    heatmap_dir = data_dir / "heatmap_data"
    heatmap_dir.mkdir(parents=True, exist_ok=True)
    
    datasets = {}
    
    # 1. Gene expression-like matrix for pathology markers
    print("🧬 Creating pathology marker expression matrix...")
    
    n_samples = 200
    n_markers = 25
    
    # Define marker categories with realistic names
    immune_markers = ['CD3', 'CD8', 'CD4', 'FOXP3', 'CD68', 'CD163', 'PD1', 'PDL1']
    proliferation_markers = ['Ki67', 'PCNA', 'MCM2', 'Cyclin_D1', 'p53']
    apoptosis_markers = ['Caspase3', 'BAX', 'BCL2', 'p21', 'TUNEL']
    structural_markers = ['CK7', 'CK20', 'Vimentin', 'SMA', 'Collagen_I', 'Fibronectin', 'E_cadherin']
    
    all_markers = immune_markers + proliferation_markers + apoptosis_markers + structural_markers
    
    # Create correlated marker expression
    marker_expression = np.zeros((n_samples, len(all_markers)))
    
    for i, marker in enumerate(all_markers):
        if marker in immune_markers:
            base_expression = np.random.lognormal(2, 1, n_samples)
        elif marker in proliferation_markers:
            base_expression = np.random.gamma(2, 2, n_samples)
        elif marker in apoptosis_markers:
            base_expression = np.random.exponential(1.5, n_samples)
        else:  # structural markers
            base_expression = np.random.beta(2, 3, n_samples) * 10
        
        marker_expression[:, i] = base_expression
    
    # Add some realistic correlations
    # Immune markers correlate with each other
    for i in range(len(immune_markers)):
        for j in range(i+1, len(immune_markers)):
            correlation_strength = np.random.uniform(0.3, 0.7)
            noise = np.random.normal(0, 0.1, n_samples)
            marker_expression[:, j] = (correlation_strength * marker_expression[:, i] + 
                                     (1-correlation_strength) * marker_expression[:, j] + noise)
    
    expression_df = pd.DataFrame(marker_expression, columns=all_markers)
    expression_df['sample_id'] = [f'S{i:03d}' for i in range(n_samples)]
    expression_df['tissue_type'] = np.random.choice(['Normal', 'Tumor', 'Metastasis'], 
                                                   n_samples, p=[0.3, 0.5, 0.2])
    expression_df['grade'] = np.random.choice([1, 2, 3], n_samples, p=[0.3, 0.5, 0.2])
    
    expression_file = heatmap_dir / "marker_expression_matrix.csv"
    expression_df.to_csv(expression_file, index=False)
    datasets['marker_expression'] = str(expression_file)
    
    # 2. Correlation matrix between clinical variables
    print("🏥 Building clinical correlation dataset...")
    
    n_patients = 300
    clinical_vars = ['age', 'tumor_size', 'lymph_nodes', 'grade', 'stage_numeric', 
                    'ki67', 'er_score', 'pr_score', 'her2_score', 'survival_months']
    
    # Define correlation structure
    correlation_matrix = np.array([
        [1.00, 0.15, 0.20, 0.25, 0.30, 0.05, -0.10, -0.05, 0.00, -0.45],  # age
        [0.15, 1.00, 0.60, 0.70, 0.75, 0.50, -0.20, -0.15, 0.25, -0.65],  # tumor_size
        [0.20, 0.60, 1.00, 0.55, 0.80, 0.35, -0.30, -0.25, 0.15, -0.70],  # lymph_nodes
        [0.25, 0.70, 0.55, 1.00, 0.65, 0.60, -0.40, -0.35, 0.30, -0.55],  # grade
        [0.30, 0.75, 0.80, 0.65, 1.00, 0.45, -0.35, -0.30, 0.20, -0.75],  # stage
        [0.05, 0.50, 0.35, 0.60, 0.45, 1.00, -0.50, -0.45, 0.10, -0.40],  # ki67
        [-0.10, -0.20, -0.30, -0.40, -0.35, -0.50, 1.00, 0.70, -0.15, 0.35],  # er_score
        [-0.05, -0.15, -0.25, -0.35, -0.30, -0.45, 0.70, 1.00, -0.10, 0.30],  # pr_score
        [0.00, 0.25, 0.15, 0.30, 0.20, 0.10, -0.15, -0.10, 1.00, -0.25],  # her2_score
        [-0.45, -0.65, -0.70, -0.55, -0.75, -0.40, 0.35, 0.30, -0.25, 1.00]   # survival
    ])
    
    # Generate correlated clinical data
    mvn_data = np.random.multivariate_normal(
        mean=np.zeros(len(clinical_vars)),
        cov=correlation_matrix,
        size=n_patients
    )
    
    # Transform to realistic ranges
    clinical_data = {
        'patient_id': [f'P{i:03d}' for i in range(n_patients)],
        'age': np.clip(65 + mvn_data[:, 0] * 12, 25, 90).astype(int),
        'tumor_size': np.maximum(5, 25 + mvn_data[:, 1] * 15),
        'lymph_nodes': np.maximum(0, 2 + mvn_data[:, 2] * 4).astype(int),
        'grade': np.clip(2 + mvn_data[:, 3] * 0.8, 1, 3).astype(int),
        'stage_numeric': np.clip(2 + mvn_data[:, 4] * 1.2, 1, 4).astype(int),
        'ki67': np.clip(20 + mvn_data[:, 5] * 25, 0, 100),
        'er_score': np.clip(50 + mvn_data[:, 6] * 40, 0, 100),
        'pr_score': np.clip(40 + mvn_data[:, 7] * 35, 0, 100),
        'her2_score': np.clip(1.5 + mvn_data[:, 8] * 1, 0, 3),
        'survival_months': np.maximum(3, 36 + mvn_data[:, 9] * 24)
    }
    
    clinical_df = pd.DataFrame(clinical_data)
    clinical_file = heatmap_dir / "clinical_correlations.csv"
    clinical_df.to_csv(clinical_file, index=False)
    datasets['clinical_correlations'] = str(clinical_file)
    
    # 3. Time-series heatmap data (treatments over time)
    print("⏰ Creating treatment response time-series...")
    
    treatments = ['Surgery', 'Chemotherapy', 'Radiation', 'Immunotherapy', 'Hormone_therapy']
    time_points = ['Baseline', '1_month', '3_months', '6_months', '12_months', '24_months']
    
    timeseries_data = []
    for patient in range(100):
        for treatment in treatments:
            for time_point in time_points:
                # Simulate treatment response over time
                time_index = time_points.index(time_point)
                base_response = np.random.normal(50, 15)
                
                # Different treatments have different response patterns
                if treatment == 'Surgery':
                    response = base_response + (time_index * 10) + np.random.normal(0, 5)
                elif treatment == 'Chemotherapy':
                    response = base_response + (20 - time_index * 2) + np.random.normal(0, 8)
                elif treatment == 'Radiation':
                    response = base_response + (15 - time_index * 1.5) + np.random.normal(0, 6)
                elif treatment == 'Immunotherapy':
                    response = base_response + (time_index * 5) + np.random.normal(0, 10)
                else:  # Hormone therapy
                    response = base_response + (time_index * 3) + np.random.normal(0, 7)
                
                timeseries_data.append({
                    'patient_id': f'T{patient:03d}',
                    'treatment': treatment,
                    'time_point': time_point,
                    'response_score': max(0, min(100, response)),
                    'toxicity_score': np.random.beta(1, 3) * 50
                })
    
    timeseries_df = pd.DataFrame(timeseries_data)
    timeseries_file = heatmap_dir / "treatment_timeseries.csv"
    timeseries_df.to_csv(timeseries_file, index=False)
    datasets['treatment_timeseries'] = str(timeseries_file)
    
    # Save dataset metadata
    metadata = {
        'creation_info': {
            'date': pd.Timestamp.now().isoformat(),
            'purpose': 'Heatmap and correlation visualization examples',
            'seed': 456
        },
        'datasets': {
            'marker_expression': {
                'description': 'Pathology marker expression matrix',
                'dimensions': f"{n_samples} samples × {len(all_markers)} markers",
                'marker_categories': {
                    'immune': immune_markers,
                    'proliferation': proliferation_markers,
                    'apoptosis': apoptosis_markers,
                    'structural': structural_markers
                }
            },
            'clinical_correlations': {
                'description': 'Correlated clinical variables',
                'n_patients': n_patients,
                'variables': clinical_vars,
                'correlation_structure': 'Realistic clinical relationships'
            },
            'treatment_timeseries': {
                'description': 'Treatment response over time',
                'treatments': treatments,
                'time_points': time_points,
                'n_patients': 100
            }
        }
    }
    
    metadata_file = heatmap_dir / "heatmap_metadata.json"
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"🎯 Heatmap datasets ready! {len(datasets)} datasets created")
    print(f"📁 Data location: {heatmap_dir.absolute()}")
    
    return heatmap_dir, datasets, metadata

# Generate heatmap and correlation datasets
heatmap_data_dir, available_heatmap_datasets, heatmap_metadata = create_heatmap_datasets()

print("🗺️ Available heatmap datasets:")
for name, path in available_heatmap_datasets.items():
    print(f"  • {name}: {Path(path).name}")

print("\n🔥 Ready for heatmap and correlation visualization!")

# Heatmaps and Correlation Analysis for Digital Pathology

This notebook focuses on advanced correlation analysis and heatmap visualizations specifically designed for digital pathology research. You'll learn to create publication-quality heatmaps and perform comprehensive correlation studies.

## Learning Objectives
1. Create correlation matrices for pathology features
2. Generate advanced heatmaps with clustering
3. Perform partial correlation analysis
4. Visualize feature importance and biomarker relationships
5. Create interactive correlation plots
6. Handle missing data in correlation analysis

## Prerequisites
- Completed previous visualization notebooks
- Understanding of correlation concepts
- Basic knowledge of pathology feature types

## 1. Environment Setup and Data Preparation

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import pearsonr, spearmanr, kendalltau
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

# Configure plotting
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
sns.set_style("whitegrid")

print("✅ Libraries imported successfully!")

In [ ]:
# Create comprehensive pathology dataset for correlation analysis
np.random.seed(42)
n_patients = 300

# Generate correlated pathology features
# Nuclear morphology features (these should be correlated)
nuclear_area = np.random.normal(150, 30, n_patients)
nuclear_perimeter = 2 * np.sqrt(np.pi * nuclear_area) + np.random.normal(0, 5, n_patients)
nuclear_compactness = (4 * np.pi * nuclear_area) / (nuclear_perimeter ** 2) + np.random.normal(0, 0.1, n_patients)
nuclear_eccentricity = np.random.uniform(0.1, 0.9, n_patients)

# Chromatin texture features (partially correlated)
chromatin_density = np.random.normal(0.6, 0.15, n_patients)
chromatin_homogeneity = 1 - chromatin_density * 0.3 + np.random.normal(0, 0.1, n_patients)
chromatin_contrast = chromatin_density * 1.5 + np.random.normal(0, 0.2, n_patients)

# Cell proliferation markers (should correlate with malignancy)
mitotic_count = np.random.poisson(3, n_patients)
ki67_index = mitotic_count * 2 + np.random.normal(15, 5, n_patients)
ki67_index = np.clip(ki67_index, 0, 100)

# Vascular features
vessel_density = np.random.normal(20, 5, n_patients)
vessel_area = vessel_density * 0.8 + np.random.normal(0, 2, n_patients)

# Inflammatory markers
lymphocyte_count = np.random.poisson(50, n_patients)
macrophage_count = lymphocyte_count * 0.3 + np.random.poisson(15, n_patients)

# Staining intensities (H&E and IHC)
hematoxylin_intensity = np.random.normal(0.6, 0.1, n_patients)
eosin_intensity = np.random.normal(0.4, 0.1, n_patients)
dab_intensity = np.random.normal(0.5, 0.15, n_patients)  # DAB for IHC

# Clinical/demographic variables
age = np.random.normal(65, 12, n_patients).astype(int)
tumor_grade = np.random.choice([1, 2, 3], n_patients, p=[0.2, 0.5, 0.3])
tumor_stage = np.random.choice([1, 2, 3, 4], n_patients, p=[0.3, 0.4, 0.2, 0.1])

# Survival data (months)
survival_time = np.random.exponential(36, n_patients)  # months
survival_time = np.where(tumor_grade == 3, survival_time * 0.6, survival_time)  # Grade 3 = worse survival

# Create DataFrame
pathology_data = pd.DataFrame({
    'patient_id': [f'P{i:04d}' for i in range(1, n_patients + 1)],
    'age': age,
    'tumor_grade': tumor_grade,
    'tumor_stage': tumor_stage,
    'nuclear_area': nuclear_area,
    'nuclear_perimeter': nuclear_perimeter,
    'nuclear_compactness': nuclear_compactness,
    'nuclear_eccentricity': nuclear_eccentricity,
    'chromatin_density': chromatin_density,
    'chromatin_homogeneity': chromatin_homogeneity,
    'chromatin_contrast': chromatin_contrast,
    'mitotic_count': mitotic_count,
    'ki67_index': ki67_index,
    'vessel_density': vessel_density,
    'vessel_area': vessel_area,
    'lymphocyte_count': lymphocyte_count,
    'macrophage_count': macrophage_count,
    'hematoxylin_intensity': hematoxylin_intensity,
    'eosin_intensity': eosin_intensity,
    'dab_intensity': dab_intensity,
    'survival_time': survival_time
})

# Add some missing values to make it realistic
missing_indices = np.random.choice(pathology_data.index, size=int(0.05 * len(pathology_data)), replace=False)
pathology_data.loc[missing_indices, 'ki67_index'] = np.nan

missing_indices_2 = np.random.choice(pathology_data.index, size=int(0.03 * len(pathology_data)), replace=False)
pathology_data.loc[missing_indices_2, 'dab_intensity'] = np.nan

print(f"✅ Pathology dataset created with {len(pathology_data)} patients")
print(f"Features: {pathology_data.shape[1]} columns")
print(f"Missing values: {pathology_data.isnull().sum().sum()} total")
pathology_data.head()

## 2. Basic Correlation Matrix and Heatmap

Start with fundamental correlation analysis and visualization.

In [ ]:
# Select numerical features for correlation analysis
numerical_features = pathology_data.select_dtypes(include=[np.number]).columns.tolist()
numerical_features.remove('patient_id') if 'patient_id' in numerical_features else None

print(f"Analyzing correlations for {len(numerical_features)} numerical features:")
for i, feature in enumerate(numerical_features, 1):
    print(f"{i:2d}. {feature}")

# Calculate correlation matrix
correlation_matrix = pathology_data[numerical_features].corr()

print(f"\n✅ Correlation matrix calculated ({correlation_matrix.shape})")

In [ ]:
# Create basic correlation heatmap
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Basic heatmap
sns.heatmap(correlation_matrix, 
            annot=True, 
            cmap='RdBu_r', 
            center=0, 
            vmin=-1, vmax=1,
            square=True,
            linewidths=0.5,
            cbar_kws={"shrink": 0.8},
            annot_kws={'size': 8},
            ax=axes[0])
axes[0].set_title('Complete Correlation Matrix', fontsize=14, fontweight='bold')
axes[0].tick_params(axis='x', rotation=45)
axes[0].tick_params(axis='y', rotation=0)

# Focus on strong correlations only (|r| > 0.3)
strong_corr_mask = np.abs(correlation_matrix) < 0.3
masked_correlation = correlation_matrix.copy()
masked_correlation[strong_corr_mask] = 0

sns.heatmap(masked_correlation, 
            annot=True, 
            cmap='RdBu_r', 
            center=0, 
            vmin=-1, vmax=1,
            square=True,
            linewidths=0.5,
            cbar_kws={"shrink": 0.8},
            annot_kws={'size': 8},
            ax=axes[1])
axes[1].set_title('Strong Correlations Only (|r| ≥ 0.3)', fontsize=14, fontweight='bold')
axes[1].tick_params(axis='x', rotation=45)
axes[1].tick_params(axis='y', rotation=0)

plt.tight_layout()
plt.show()

# Print strongest correlations
print("=== STRONGEST CORRELATIONS ===")
# Get upper triangle to avoid duplicates
upper_triangle = correlation_matrix.where(np.triu(np.ones_like(correlation_matrix, dtype=bool), k=1))
strong_correlations = upper_triangle.stack().sort_values(key=abs, ascending=False)

print("Top 10 strongest correlations:")
for i, (pair, corr_value) in enumerate(strong_correlations.head(10).items(), 1):
    feature1, feature2 = pair
    strength = "Very Strong" if abs(corr_value) > 0.7 else "Strong" if abs(corr_value) > 0.5 else "Moderate"
    direction = "Positive" if corr_value > 0 else "Negative"
    print(f"{i:2d}. {feature1} ↔ {feature2}: {corr_value:.3f} ({strength} {direction})")

## 3. Advanced Clustered Heatmap

Create heatmaps with hierarchical clustering to group related features.

In [ ]:
# Create clustered heatmap using hierarchical clustering
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform

# Calculate distance matrix (1 - |correlation|)
distance_matrix = 1 - np.abs(correlation_matrix)

# Perform hierarchical clustering
linkage_matrix = linkage(squareform(distance_matrix), method='average')

# Create clustered heatmap
fig, axes = plt.subplots(2, 2, figsize=(20, 16))

# 1. Dendrogram
dendrogram(linkage_matrix, labels=correlation_matrix.columns, 
           orientation='left', ax=axes[0, 0])
axes[0, 0].set_title('Feature Dendrogram', fontsize=14, fontweight='bold')

# 2. Clustered correlation matrix
from scipy.cluster.hierarchy import leaves_list
cluster_order = leaves_list(linkage_matrix)
clustered_corr = correlation_matrix.iloc[cluster_order, cluster_order]

sns.heatmap(clustered_corr, 
            annot=True, 
            cmap='RdBu_r', 
            center=0,
            vmin=-1, vmax=1,
            square=True,
            linewidths=0.5,
            annot_kws={'size': 7},
            ax=axes[0, 1])
axes[0, 1].set_title('Clustered Correlation Matrix', fontsize=14, fontweight='bold')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].tick_params(axis='y', rotation=0)

# 3. Feature groups identification
n_clusters = 4  # Number of feature groups
cluster_labels = fcluster(linkage_matrix, n_clusters, criterion='maxclust')

# Create feature groups
feature_groups = {}
for i, feature in enumerate(correlation_matrix.columns):
    cluster_id = cluster_labels[i]
    if cluster_id not in feature_groups:
        feature_groups[cluster_id] = []
    feature_groups[cluster_id].append(feature)

print("=== IDENTIFIED FEATURE GROUPS ===")
group_colors = ['red', 'blue', 'green', 'orange', 'purple']
for group_id, features in feature_groups.items():
    print(f"\nGroup {group_id} ({len(features)} features):")
    for feature in features:
        print(f"  • {feature}")

# 4. Group-wise correlation analysis
group_correlations = []
for group_id, features in feature_groups.items():
    if len(features) > 1:
        group_corr_matrix = correlation_matrix.loc[features, features]
        # Calculate mean within-group correlation
        upper_tri = group_corr_matrix.where(np.triu(np.ones_like(group_corr_matrix, dtype=bool), k=1))
        mean_corr = upper_tri.stack().mean()
        group_correlations.append({'Group': group_id, 'Mean_Correlation': mean_corr, 'Features': features})

# Visualize group correlations
if group_correlations:
    group_df = pd.DataFrame(group_correlations)
    axes[1, 0].bar(range(len(group_df)), group_df['Mean_Correlation'], color=group_colors[:len(group_df)])
    axes[1, 0].set_xlabel('Feature Group')
    axes[1, 0].set_ylabel('Mean Within-Group Correlation')
    axes[1, 0].set_title('Within-Group Correlation Strength')
    axes[1, 0].set_xticks(range(len(group_df)))
    axes[1, 0].set_xticklabels([f'Group {g}' for g in group_df['Group']])

# 5. Cross-group correlation heatmap
cross_group_corr = np.zeros((len(feature_groups), len(feature_groups)))
group_names = []

for i, (group1_id, features1) in enumerate(feature_groups.items()):
    group_names.append(f'Group {group1_id}')
    for j, (group2_id, features2) in enumerate(feature_groups.items()):
        if i != j:  # Different groups
            cross_corr_values = []
            for f1 in features1:
                for f2 in features2:
                    cross_corr_values.append(abs(correlation_matrix.loc[f1, f2]))
            cross_group_corr[i, j] = np.mean(cross_corr_values) if cross_corr_values else 0

sns.heatmap(cross_group_corr, 
            annot=True, 
            cmap='YlOrRd',
            xticklabels=group_names,
            yticklabels=group_names,
            square=True,
            ax=axes[1, 1])
axes[1, 1].set_title('Cross-Group Correlation Strength', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 4. Feature-Specific Correlation Analysis

Analyze correlations for different types of pathology features.

In [ ]:
# Define feature categories based on biological relevance
feature_categories = {
    'Nuclear Morphology': ['nuclear_area', 'nuclear_perimeter', 'nuclear_compactness', 'nuclear_eccentricity'],
    'Chromatin Features': ['chromatin_density', 'chromatin_homogeneity', 'chromatin_contrast'],
    'Proliferation Markers': ['mitotic_count', 'ki67_index'],
    'Vascular Features': ['vessel_density', 'vessel_area'],
    'Immune Markers': ['lymphocyte_count', 'macrophage_count'],
    'Staining Intensities': ['hematoxylin_intensity', 'eosin_intensity', 'dab_intensity'],
    'Clinical Variables': ['age', 'tumor_grade', 'tumor_stage', 'survival_time']
}

# Create category-based analysis
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
axes = axes.flatten()

category_stats = []

for i, (category, features) in enumerate(feature_categories.items()):
    if i < len(axes):
        # Filter features that exist in dataset
        available_features = [f for f in features if f in pathology_data.columns]
        
        if len(available_features) > 1:
            # Calculate correlation matrix for this category
            cat_corr = pathology_data[available_features].corr()
            
            # Create heatmap
            sns.heatmap(cat_corr, 
                       annot=True, 
                       cmap='RdBu_r', 
                       center=0,
                       vmin=-1, vmax=1,
                       square=True,
                       linewidths=1,
                       annot_kws={'size': 10},
                       ax=axes[i])
            axes[i].set_title(f'{category}\n({len(available_features)} features)', 
                             fontsize=12, fontweight='bold')
            axes[i].tick_params(axis='x', rotation=45)
            axes[i].tick_params(axis='y', rotation=0)
            
            # Calculate statistics
            upper_tri = cat_corr.where(np.triu(np.ones_like(cat_corr, dtype=bool), k=1))
            correlations = upper_tri.stack()
            
            category_stats.append({
                'Category': category,
                'N_Features': len(available_features),
                'Mean_Correlation': correlations.mean(),
                'Max_Correlation': correlations.max(),
                'Min_Correlation': correlations.min(),
                'Strong_Correlations': sum(abs(correlations) > 0.5)
            })

# Remove empty subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

# Display category statistics
if category_stats:
    stats_df = pd.DataFrame(category_stats)
    print("=== CATEGORY-WISE CORRELATION STATISTICS ===")
    print(stats_df.round(3).to_string(index=False))
    
    # Visualize category statistics
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Mean correlations by category
    axes[0].barh(stats_df['Category'], stats_df['Mean_Correlation'], color='skyblue')
    axes[0].set_xlabel('Mean Correlation')
    axes[0].set_title('Average Correlation Strength by Feature Category')
    axes[0].axvline(x=0, color='black', linestyle='-', alpha=0.3)
    
    # Number of strong correlations
    axes[1].barh(stats_df['Category'], stats_df['Strong_Correlations'], color='coral')
    axes[1].set_xlabel('Number of Strong Correlations (|r| > 0.5)')
    axes[1].set_title('Strong Correlations Count by Category')
    
    plt.tight_layout()
    plt.show()

## 5. Partial Correlation Analysis

Analyze correlations while controlling for confounding variables.

In [ ]:
# Partial correlation analysis
from scipy.stats import pearsonr

def partial_correlation(df, x, y, control_vars):
    """
    Calculate partial correlation between x and y, controlling for control_vars
    """
    # Remove rows with missing values
    analysis_data = df[[x, y] + control_vars].dropna()
    
    if len(analysis_data) < 10:  # Need sufficient data
        return np.nan, np.nan
    
    # Standardize all variables
    scaler = StandardScaler()
    standardized_data = scaler.fit_transform(analysis_data)
    standardized_df = pd.DataFrame(standardized_data, columns=[x, y] + control_vars)
    
    # Calculate residuals after regressing out control variables
    from sklearn.linear_model import LinearRegression
    
    reg_model = LinearRegression()
    
    # Residuals for x after controlling for control_vars
    if control_vars:
        reg_model.fit(standardized_df[control_vars], standardized_df[x])
        x_residuals = standardized_df[x] - reg_model.predict(standardized_df[control_vars])
        
        # Residuals for y after controlling for control_vars
        reg_model.fit(standardized_df[control_vars], standardized_df[y])
        y_residuals = standardized_df[y] - reg_model.predict(standardized_df[control_vars])
    else:
        x_residuals = standardized_df[x]
        y_residuals = standardized_df[y]
    
    # Calculate correlation of residuals
    return pearsonr(x_residuals, y_residuals)

# Perform partial correlation analysis
print("=== PARTIAL CORRELATION ANALYSIS ===")
print("Controlling for age, tumor_grade, and tumor_stage")

# Define pairs to analyze
correlation_pairs = [
    ('nuclear_area', 'mitotic_count'),
    ('chromatin_density', 'ki67_index'),
    ('vessel_density', 'lymphocyte_count'),
    ('nuclear_compactness', 'survival_time'),
    ('chromatin_contrast', 'tumor_grade')
]

control_variables = ['age', 'tumor_grade', 'tumor_stage']

partial_corr_results = []

print(f"\n{'Pair':<40} {'Raw Corr':<12} {'Partial Corr':<12} {'P-value':<12} {'Change'}")
print("-" * 85)

for x, y in correlation_pairs:
    if x in pathology_data.columns and y in pathology_data.columns:
        # Raw correlation
        raw_data = pathology_data[[x, y]].dropna()
        if len(raw_data) > 0:
            raw_corr, raw_p = pearsonr(raw_data[x], raw_data[y])
        else:
            raw_corr, raw_p = np.nan, np.nan
        
        # Partial correlation
        partial_corr, partial_p = partial_correlation(pathology_data, x, y, control_variables)
        
        # Calculate change
        change = partial_corr - raw_corr if not (np.isnan(raw_corr) or np.isnan(partial_corr)) else np.nan
        
        partial_corr_results.append({
            'Variable_1': x,
            'Variable_2': y,
            'Raw_Correlation': raw_corr,
            'Partial_Correlation': partial_corr,
            'P_value': partial_p,
            'Change': change
        })
        
        print(f"{x} ↔ {y:<25} {raw_corr:>8.3f}   {partial_corr:>8.3f}   {partial_p:>8.3f}   {change:>+6.3f}")

# Visualize partial vs raw correlations
partial_df = pd.DataFrame(partial_corr_results).dropna()

if not partial_df.empty:
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Scatter plot: Raw vs Partial correlations
    axes[0].scatter(partial_df['Raw_Correlation'], partial_df['Partial_Correlation'], 
                   s=100, alpha=0.7, color='purple')
    axes[0].plot([-1, 1], [-1, 1], 'r--', alpha=0.5, label='No change line')
    axes[0].set_xlabel('Raw Correlation')
    axes[0].set_ylabel('Partial Correlation')
    axes[0].set_title('Raw vs Partial Correlations')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Add labels for points
    for i, row in partial_df.iterrows():
        axes[0].annotate(f"{row['Variable_1'][:6]}-{row['Variable_2'][:6]}", 
                        (row['Raw_Correlation'], row['Partial_Correlation']),
                        xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    # Bar plot: Changes in correlation
    pair_labels = [f"{row['Variable_1'][:8]}-{row['Variable_2'][:8]}" for _, row in partial_df.iterrows()]
    colors = ['red' if change < 0 else 'blue' for change in partial_df['Change']]
    
    axes[1].bar(range(len(partial_df)), partial_df['Change'], color=colors, alpha=0.7)
    axes[1].set_xlabel('Variable Pairs')
    axes[1].set_ylabel('Change in Correlation')
    axes[1].set_title('Change in Correlation After Controlling for Confounders')
    axes[1].set_xticks(range(len(partial_df)))
    axes[1].set_xticklabels(pair_labels, rotation=45, ha='right')
    axes[1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary
    mean_change = partial_df['Change'].mean()
    print(f"\n✅ Partial correlation analysis completed!")
    print(f"Average change in correlation after controlling for confounders: {mean_change:+.3f}")
    print(f"Largest decrease: {partial_df['Change'].min():.3f}")
    print(f"Largest increase: {partial_df['Change'].max():.3f}")

## 6. Interactive Correlation Explorer

Create an interactive-style correlation explorer with detailed insights.

In [ ]:
# Interactive correlation explorer (static version for notebook)
print("=== INTERACTIVE CORRELATION EXPLORER ===")

def explore_correlation(feature1, feature2, data):
    """Detailed correlation analysis between two features"""
    
    # Check if features exist
    if feature1 not in data.columns or feature2 not in data.columns:
        print(f"❌ One or both features not found in dataset")
        return
    
    # Get clean data
    clean_data = data[[feature1, feature2]].dropna()
    
    if len(clean_data) < 10:
        print(f"❌ Insufficient data points ({len(clean_data)}) for analysis")
        return
    
    # Calculate correlations
    pearson_r, pearson_p = pearsonr(clean_data[feature1], clean_data[feature2])
    spearman_r, spearman_p = spearmanr(clean_data[feature1], clean_data[feature2])
    kendall_r, kendall_p = kendalltau(clean_data[feature1], clean_data[feature2])
    
    # Create detailed analysis plot
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Scatter plot with regression line
    axes[0, 0].scatter(clean_data[feature1], clean_data[feature2], alpha=0.6, s=50)
    
    # Add regression line
    z = np.polyfit(clean_data[feature1], clean_data[feature2], 1)
    p = np.poly1d(z)
    axes[0, 0].plot(clean_data[feature1], p(clean_data[feature1]), "r--", alpha=0.8)
    
    axes[0, 0].set_xlabel(feature1.replace('_', ' ').title())
    axes[0, 0].set_ylabel(feature2.replace('_', ' ').title())
    axes[0, 0].set_title(f'Scatter Plot: {feature1} vs {feature2}')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Add correlation info
    axes[0, 0].text(0.05, 0.95, f'Pearson r = {pearson_r:.3f}\np = {pearson_p:.4f}', 
                   transform=axes[0, 0].transAxes, verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='wheat'))
    
    # Distribution plots
    axes[0, 1].hist(clean_data[feature1], bins=20, alpha=0.7, color='blue', label=feature1)
    axes[0, 1].axvline(clean_data[feature1].mean(), color='blue', linestyle='--', alpha=0.8)
    axes[0, 1].set_xlabel(feature1.replace('_', ' ').title())
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].set_title(f'Distribution of {feature1}')
    axes[0, 1].legend()
    
    axes[1, 0].hist(clean_data[feature2], bins=20, alpha=0.7, color='red', label=feature2)
    axes[1, 0].axvline(clean_data[feature2].mean(), color='red', linestyle='--', alpha=0.8)
    axes[1, 0].set_xlabel(feature2.replace('_', ' ').title())
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].set_title(f'Distribution of {feature2}')
    axes[1, 0].legend()
    
    # Correlation comparison table
    correlation_data = {
        'Method': ['Pearson', 'Spearman', 'Kendall'],
        'Correlation': [pearson_r, spearman_r, kendall_r],
        'P-value': [pearson_p, spearman_p, kendall_p],
        'Significance': ['***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else 'ns' 
                        for p in [pearson_p, spearman_p, kendall_p]]
    }
    
    corr_df = pd.DataFrame(correlation_data)
    
    # Create table plot
    axes[1, 1].axis('tight')
    axes[1, 1].axis('off')
    table = axes[1, 1].table(cellText=corr_df.round(4).values, colLabels=corr_df.columns,
                           cellLoc='center', loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1.2, 2)
    axes[1, 1].set_title('Correlation Analysis Results')
    
    plt.suptitle(f'Detailed Correlation Analysis: {feature1} ↔ {feature2}', 
                fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Print detailed results
    print(f"\n📊 DETAILED ANALYSIS: {feature1} ↔ {feature2}")
    print(f"Data points: {len(clean_data)}")
    print(f"Missing values: {len(data) - len(clean_data)}")
    print()
    print("CORRELATION RESULTS:")
    for _, row in corr_df.iterrows():
        strength = "Very Strong" if abs(row['Correlation']) > 0.7 else "Strong" if abs(row['Correlation']) > 0.5 else "Moderate" if abs(row['Correlation']) > 0.3 else "Weak"
        direction = "Positive" if row['Correlation'] > 0 else "Negative"
        print(f"• {row['Method']}: r = {row['Correlation']:.3f} (p = {row['P-value']:.4f}) [{strength} {direction}] {row['Significance']}")
    
    print()
    print("DESCRIPTIVE STATISTICS:")
    print(f"• {feature1}: Mean = {clean_data[feature1].mean():.2f}, SD = {clean_data[feature1].std():.2f}")
    print(f"• {feature2}: Mean = {clean_data[feature2].mean():.2f}, SD = {clean_data[feature2].std():.2f}")
    
    return pearson_r, pearson_p

# Demonstrate interactive exploration with key feature pairs
interesting_pairs = [
    ('nuclear_area', 'nuclear_perimeter'),
    ('mitotic_count', 'ki67_index'),
    ('chromatin_density', 'chromatin_contrast'),
    ('tumor_grade', 'survival_time')
]

print("🔍 Exploring interesting feature correlations...")
print("=" * 80)

for feature1, feature2 in interesting_pairs:
    if feature1 in pathology_data.columns and feature2 in pathology_data.columns:
        explore_correlation(feature1, feature2, pathology_data)
        print("\n" + "="*80 + "\n")

## 7. Missing Data Impact on Correlations

Analyze how missing data affects correlation estimates.

In [ ]:
# Missing data impact analysis
print("=== MISSING DATA IMPACT ANALYSIS ===")

# Check current missing data patterns
missing_summary = pathology_data.isnull().sum()
missing_features = missing_summary[missing_summary > 0]

if len(missing_features) > 0:
    print("Features with missing data:")
    for feature, count in missing_features.items():
        percentage = (count / len(pathology_data)) * 100
        print(f"• {feature}: {count} missing ({percentage:.1f}%)")
    
    # Analyze correlation impact
    print("\n=== CORRELATION IMPACT ANALYSIS ===")
    
    # Compare correlations with different missing data handling strategies
    strategies = {
        'Listwise Deletion': 'listwise',
        'Pairwise Deletion': 'pairwise',
        'Mean Imputation': 'mean_impute'
    }
    
    comparison_results = []
    
    # Select features with some missing data for analysis
    test_features = ['nuclear_area', 'ki67_index', 'chromatin_density', 'dab_intensity']
    test_features = [f for f in test_features if f in pathology_data.columns]
    
    for strategy_name, strategy_code in strategies.items():
        if strategy_code == 'listwise':
            # Use complete cases only
            analysis_data = pathology_data[test_features].dropna()
            
        elif strategy_code == 'pairwise':
            # Calculate correlations pairwise (built-in pandas behavior)
            analysis_data = pathology_data[test_features]
            
        elif strategy_code == 'mean_impute':
            # Replace missing values with mean
            analysis_data = pathology_data[test_features].copy()
            for col in analysis_data.columns:
                if analysis_data[col].isnull().sum() > 0:
                    analysis_data[col].fillna(analysis_data[col].mean(), inplace=True)
        
        # Calculate correlation matrix
        if strategy_code == 'pairwise':
            corr_matrix = pathology_data[test_features].corr(method='pearson')
        else:
            corr_matrix = analysis_data.corr(method='pearson')
        
        # Store results
        comparison_results.append({
            'Strategy': strategy_name,
            'N_observations': len(analysis_data) if strategy_code != 'pairwise' else 'Varies',
            'Mean_correlation': corr_matrix.values[np.triu_indices_from(corr_matrix.values, k=1)].mean(),
            'Correlation_matrix': corr_matrix
        })
        
        print(f"\n{strategy_name}:")
        print(f"  Effective sample size: {len(analysis_data) if strategy_code != 'pairwise' else 'Pairwise variable'}")
        print(f"  Mean absolute correlation: {np.abs(corr_matrix.values[np.triu_indices_from(corr_matrix.values, k=1)]).mean():.3f}")
    
    # Visualize comparison
    fig, axes = plt.subplots(1, len(comparison_results), figsize=(5 * len(comparison_results), 6))
    if len(comparison_results) == 1:
        axes = [axes]
    
    for i, result in enumerate(comparison_results):
        sns.heatmap(result['Correlation_matrix'], 
                   annot=True, 
                   cmap='RdBu_r', 
                   center=0,
                   vmin=-1, vmax=1,
                   square=True,
                   ax=axes[i])
        axes[i].set_title(f"{result['Strategy']}\nN = {result['N_observations']}")
    
    plt.tight_layout()
    plt.show()
    
    # Statistical comparison
    print("\n=== STRATEGY COMPARISON ===")
    print("Differences in correlation estimates:")
    
    baseline_corr = comparison_results[0]['Correlation_matrix']  # Listwise as baseline
    
    for i, result in enumerate(comparison_results[1:], 1):
        current_corr = result['Correlation_matrix']
        
        # Calculate differences
        diff_matrix = current_corr - baseline_corr
        mean_diff = np.abs(diff_matrix.values[np.triu_indices_from(diff_matrix.values, k=1)]).mean()
        max_diff = np.abs(diff_matrix.values).max()
        
        print(f"\n{result['Strategy']} vs Listwise Deletion:")
        print(f"  Mean absolute difference: {mean_diff:.4f}")
        print(f"  Maximum absolute difference: {max_diff:.4f}")
        
        if mean_diff > 0.1:
            print(f"  ⚠️  Large differences detected - consider impact on analysis!")
        elif mean_diff > 0.05:
            print(f"  ⚡ Moderate differences - review carefully")
        else:
            print(f"  ✅ Small differences - strategies are comparable")

else:
    print("✅ No missing data found in the current dataset")

# Sensitivity analysis - artificially introduce missing data
print("\n=== SENSITIVITY ANALYSIS ===")
print("Analyzing correlation stability with increasing missing data...")

# Create a copy for sensitivity analysis
sensitivity_data = pathology_data.copy()

# Select a key correlation pair
test_feature1, test_feature2 = 'nuclear_area', 'mitotic_count'
if test_feature1 in sensitivity_data.columns and test_feature2 in sensitivity_data.columns:
    
    # Original correlation
    original_corr, _ = pearsonr(sensitivity_data[test_feature1].dropna(), 
                               sensitivity_data[test_feature2][sensitivity_data[test_feature1].notna()])
    
    missing_percentages = [0, 5, 10, 15, 20, 25, 30]
    correlation_changes = []
    
    np.random.seed(42)  # For reproducibility
    
    for missing_pct in missing_percentages:
        if missing_pct == 0:
            correlation_changes.append(0)
            continue
            
        # Introduce missing data randomly
        test_data = sensitivity_data[[test_feature1, test_feature2]].copy()
        n_missing = int(len(test_data) * missing_pct / 100)
        missing_indices = np.random.choice(test_data.index, n_missing, replace=False)
        test_data.loc[missing_indices, test_feature1] = np.nan
        
        # Calculate correlation with missing data (pairwise deletion)
        clean_data = test_data.dropna()
        if len(clean_data) > 10:
            new_corr, _ = pearsonr(clean_data[test_feature1], clean_data[test_feature2])
            correlation_changes.append(abs(new_corr - original_corr))
        else:
            correlation_changes.append(np.nan)
    
    # Plot sensitivity analysis
    plt.figure(figsize=(10, 6))
    plt.plot(missing_percentages, correlation_changes, 'bo-', markersize=8, linewidth=2)
    plt.xlabel('Missing Data Percentage (%)')
    plt.ylabel('Absolute Change in Correlation')
    plt.title(f'Correlation Stability: {test_feature1} ↔ {test_feature2}')
    plt.grid(True, alpha=0.3)
    
    # Add threshold lines
    plt.axhline(y=0.05, color='orange', linestyle='--', alpha=0.7, label='5% change threshold')
    plt.axhline(y=0.1, color='red', linestyle='--', alpha=0.7, label='10% change threshold')
    plt.legend()
    
    # Annotate points
    for i, (pct, change) in enumerate(zip(missing_percentages, correlation_changes)):
        if not np.isnan(change):
            plt.annotate(f'{change:.3f}', (pct, change), textcoords="offset points", 
                        xytext=(0,10), ha='center', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Original correlation: {original_corr:.3f}")
    print("Stability assessment:")
    for pct, change in zip(missing_percentages[1:], correlation_changes[1:]):
        if not np.isnan(change):
            stability = "Stable" if change < 0.05 else "Moderate" if change < 0.1 else "Unstable"
            print(f"  {pct:2d}% missing: Δr = {change:.3f} ({stability})")

## 8. Auto-Validation Tests

In [ ]:
# Auto-validation tests for correlation and heatmap analysis
print("=== AUTO-VALIDATION TESTS ===")

# Test 1: Correlation matrix properties
correlation_matrix = pathology_data.select_dtypes(include=[np.number]).corr()
assert correlation_matrix.shape[0] == correlation_matrix.shape[1], "❌ Correlation matrix not square"
assert np.allclose(np.diag(correlation_matrix), 1.0, atol=1e-10), "❌ Diagonal elements not equal to 1"
assert np.allclose(correlation_matrix.values, correlation_matrix.values.T, atol=1e-10), "❌ Correlation matrix not symmetric"
print("✅ Test 1 passed: Correlation matrix properties are correct")

# Test 2: Correlation range validation
correlation_values = correlation_matrix.values[np.triu_indices_from(correlation_matrix.values, k=1)]
assert np.all((-1 <= correlation_values) & (correlation_values <= 1)), "❌ Correlation values outside valid range"
print("✅ Test 2 passed: All correlation values in valid range [-1, 1]")

# Test 3: Statistical significance calculation
test_corr, test_p = pearsonr(pathology_data['age'], pathology_data['nuclear_area'])
assert 0 <= test_p <= 1, "❌ P-value outside valid range"
assert not np.isnan(test_corr), "❌ Correlation calculation returned NaN"
print("✅ Test 3 passed: Statistical calculations are valid")

# Test 4: Missing data handling
data_with_missing = pathology_data.copy()
data_with_missing.loc[:10, 'nuclear_area'] = np.nan
missing_corr_matrix = data_with_missing.corr()
assert not missing_corr_matrix.isnull().all().any(), "❌ All correlations are NaN with missing data"
print("✅ Test 4 passed: Missing data handled appropriately")

# Test 5: Clustering validation
if len(correlation_matrix) > 3:
    distance_matrix = 1 - np.abs(correlation_matrix)
    linkage_matrix = linkage(squareform(distance_matrix), method='average')
    assert linkage_matrix.shape[0] == correlation_matrix.shape[0] - 1, "❌ Incorrect linkage matrix dimensions"
    assert np.all(linkage_matrix[:, 2] >= 0), "❌ Negative distances in linkage matrix"
    print("✅ Test 5 passed: Hierarchical clustering works correctly")

# Test 6: Feature grouping validation
if 'feature_groups' in locals():
    total_features = sum(len(features) for features in feature_groups.values())
    assert total_features == len(correlation_matrix.columns), "❌ Feature grouping incomplete"
    print("✅ Test 6 passed: Feature grouping is complete")

# Test 7: Heatmap data integrity
heatmap_test_data = np.random.rand(5, 5)
heatmap_test_data = (heatmap_test_data + heatmap_test_data.T) / 2  # Make symmetric
np.fill_diagonal(heatmap_test_data, 1)
assert np.allclose(heatmap_test_data, heatmap_test_data.T), "❌ Test heatmap data not symmetric"
print("✅ Test 7 passed: Heatmap data structure is valid")

print("\n🎉 All validation tests passed! You've mastered correlation analysis and heatmap visualization!")

## 9. Next Steps

Congratulations! You've mastered advanced correlation analysis and heatmap visualization for digital pathology data.

**Key Skills Acquired:**
✅ Comprehensive correlation matrix analysis  
✅ Hierarchical clustering of features  
✅ Partial correlation controlling for confounders  
✅ Missing data impact assessment  
✅ Feature category analysis  
✅ Publication-quality heatmap creation  
✅ Statistical significance testing  

**In the next notebook, you'll learn about:**
- UMAP and t-SNE for dimensionality reduction
- Interactive visualization techniques
- Advanced clustering methods
- Multi-dimensional data exploration

**For Further Practice:**
- Apply these techniques to real TCGA pathology datasets
- Explore network analysis of correlated features
- Create interactive correlation dashboards
- Investigate time-series correlations in longitudinal data

**Clinical Applications:**
- Biomarker discovery and validation
- Feature selection for predictive models
- Quality control in pathology workflows
- Multi-omics data integration